In [7]:
import numpy as np
import matplotlib.pyplot as plt

# Word vector (word embedding), count models, word2vec y GloVe

En estas notas se van a estudiar los temas que involucran word vectors, es decir los modelos que tratan de representar el significado de las palabras con vectores.

Estas notas abarcan las lectures 01 y 02 del curso.

* Material introductorio:
    * Slides del curso (Lecture 1 y 2)
    * Notas del curso (Notas 1 y 2)
    * [Word2Vec Tutorial](http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/)
    * [Co-ocurrence y Skip-gram](https://medium.com/data-science-group-iitr/word-embedding-2d05d270b285)
    * [Co-ocurrence y Skip-gram 2](http://web.stanford.edu/class/cs124/lec/vectorsemantics.video.pdf)
    * Material de SVD: [1](https://davetang.org/file/Singular_Value_Decomposition_Tutorial.pdf), [2](https://web.stanford.edu/class/cs168/l/l7.pdf), [3](http://theory.stanford.edu/~tim/s15/l/l8.pdf), [4](https://web.stanford.edu/class/cs168/l/l9.pdf), [5](https://en.wikipedia.org/wiki/Singular_value_decomposition#Truncated_SVD) y [6](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.TruncatedSVD.html).
    * [Corpus nltk](https://www.nltk.org/book/ch02.html)
    * [Gensim](https://radimrehurek.com/gensim/models/keyedvectors.html#gensim.models.keyedvectors.FastTextKeyedVectors.most_similar)
    
* Papers (ordenados):
    * Efficient Estimation of Word Representations in Vector Space (original word2vec paper)
    * Distributed Representations of Words and Phrases and their Compositionality (negative sampling paper)
    * GloVe: Global Vectors for Word Representation (original GloVe paper)
    * Improving Distributional Similarity with Lessons Learned from Word Embeddings
    * Evaluation methods for unsupervised word embeddings
    * A Latent Variable Model Approach to PMI-based Word Embeddings
    * Linear Algebraic Structure of Word Senses, with Applications to Polysemy
    * On the Dimensionality of Word Embedding.
* Otras cosas encontradas por mí:
    * [Distributed representations in NN](https://www.oreilly.com/ideas/how-neural-networks-learn-distributed-representations)
    * Linguistic Regularities in Continuous Space Word Representations (opcional, también de Mikolov). Ver carpeta Vector-words
    * [Tutorial word2vec Tensorflow](https://www.tensorflow.org/tutorials/representation/word2vec) (Muy didáctico!!)
    * Libro del grupo de Hinton: "Parallel distributed processing: explorations in the microstructure of cognition". En particular, el capítulo 3 habla de representaciones distribuídas. Ver carpeta Vector-words. 
    * Don’t count, predict!A systematic comparison ofcontext-counting vs. context-predicting semantic vectors. (Distinción entre los métodos de conteo y los predictivos). Ver carpeta Vector-Words.
    
    


## Word vector

La idea es representar el significado de una palabra con un vector. De esta manera, palabras con significado "cercano" (o sea, sinónimos) estarían representadas por vectores cercanos entre ellos en este espacio.

Es decir, en la primera parte del curso queremos **representar el significado una palabra**. Para ello hay (por lo menos) dos enfoques:

1. *Denotational semantics*: Representación de las palabras con vectores one-hot.

2. *Distributional semantics*: Representación de las palabras según los diferentes contextos en que aparece. En este sentido, lo que se está diciendo implícitamente es "si vos podés elegir correctamente una palabra para usar en un contexto, entonces entendiste el significado de la palabra". Esto supongo que funcionaría bien con los slungs ("basero", "scopa tu mana", etc.) y con los hashtags de twitter.

Nosotros vamos a empezar con todo lo que es *Distributional semantics*. Para eso, a su vez, hay varias maneras de pensarlo. El assignment 1 (el cual se explica en la lecture 2 recién) se trata de contar ocurrencias de palabras y contextos y en base a ello encontrar el word embedding.

## Co-ocurrence matrix (matriz de co-ocurrencia)

La primera forma de representar el significado de una palabra es deduciéndolo a partir de su contexto. El razonamiento es: una palabra va a aparecer muy seguido al lado de ciertas palabras, dado que su significado en general se relaciona con estas. Con esto, se puede definir una matriz de coocurrencia:

A co-occurrence matrix counts how often things co-occur in some environment. Given some word $w_i$ occurring in the document, we consider the *context window* surrounding $w_i$. Supposing our fixed window size is $n$, then this is the $n$ preceding and $n$ subsequent words in that document, i.e. words $w_{i-n} \dots w_{i-1}$ and $w_{i+1} \dots w_{i+n}$. We build a *co-occurrence matrix* $M$, which is a symmetric word-by-word matrix in which $M_{ij}$ is the number of times $w_j$ appears inside $w_i$'s window.

**Example: Co-Occurrence with Fixed Window of n=1**:

Document 1: "all that glitters is not gold"

Document 2: "all is well that ends well"


|     *    | START | all | that | glitters | is   | not  | gold  | well | ends | END |
|----------|-------|-----|------|----------|------|------|-------|------|------|-----|
| START    | 0     | 2   | 0    | 0        | 0    | 0    | 0     | 0    | 0    | 0   |
| all      | 2     | 0   | 1    | 0        | 1    | 0    | 0     | 0    | 0    | 0   |
| that     | 0     | 1   | 0    | 1        | 0    | 0    | 0     | 1    | 1    | 0   |
| glitters | 0     | 0   | 1    | 0        | 1    | 0    | 0     | 0    | 0    | 0   |
| is       | 0     | 1   | 0    | 1        | 0    | 1    | 0     | 1    | 0    | 0   |
| not      | 0     | 0   | 0    | 0        | 1    | 0    | 1     | 0    | 0    | 0   |
| gold     | 0     | 0   | 0    | 0        | 0    | 1    | 0     | 0    | 0    | 1   |
| well     | 0     | 0   | 1    | 0        | 1    | 0    | 0     | 0    | 1    | 1   |
| ends     | 0     | 0   | 1    | 0        | 0    | 0    | 0     | 1    | 0    | 0   |
| END      | 0     | 0   | 0    | 0        | 0    | 0    | 1     | 1    | 0    | 0   |

**Note:** In NLP, we often add START and END tokens to represent the beginning and end of sentences, paragraphs or documents. In thise case we imagine START and END tokens encapsulating each document, e.g., "START All that glitters is not gold END", and include these tokens in our co-occurrence counts.

In [4]:
def DistinctWords(corpus):
    """ 
        Devuelve una lista con todas las palabras que aparecen en corpus, sin repetición.
        Params:
            corpus (lista de lista de strings): corpus of todos los documentos
        Return:
            corpus_words (lista de strings): lista con todas las palabras que aparecen en corpus, 
            ordenadas alfanuméricamente y sin repetición.
            num_corpus_words (entero): tamaño de la lista.
    """
    
    # Lista con las palabras del corpus sin repetición
    corpus_words = sorted(list(set([item for sublist in corpus for item in sublist])))
    
    # Tamaño de la lista
    num_corpus_words = len(corpus_words)

    return corpus_words, num_corpus_words


def GetCoocurrenceMatrix(corpus, window_size=4):
    """ 
        Calcula la matriz de co-ocurrencia de un dado corpus, usando la función anterior.
        Nota: las ventanas de las palabras en los extremos son más chicas que las de las 
        del medio, porque no tienen palabras en uno de los lados.
    
        Params:
            corpus (lista de lista de strings): corpus of todos los documentos.
            window_size (entero): tamaño de la ventana.
            
        Return:
            M: 2-D numpy array que representa la matriz de co-ocurrencia. El orden en que están las
            palabras es el mismo en que está la lista que devuelve la función DistinctWords(corpus).
            word2Ind (diccionario): Diccionario para obtener los índices de la matriz M a partir 
            de la palabra.
    """
    
    # Obtengo las palabras del corpus sin repetición
    words, num_words = DistinctWords(corpus)
    M = np.zeros((num_words,num_words))
    word2Ind = {key: value for (key,value) in zip(words,range(num_words))}
    
    # Lleno la matriz de co-ocurrencias
    for sentence in corpus:
        current_index = 0
        sentence_len = len(sentence)
        indices = [word2Ind[i] for i in sentence]
        while current_index < sentence_len:
            left  = max(current_index - window_size, 0)
            right = min(current_index + window_size + 1, sentence_len) 
            current_word = sentence[current_index]
            current_word_index = word2Ind[current_word]
            words_around = indices[left:current_index] + indices[current_index+1:right]
            
            for ind in words_around:
                M[current_word_index, ind] += 1
            
            current_index += 1
                    
    return M, word2Ind

A esto se suele hacer una reducción de la dimensionalidad con PCA o con SVD. Esto implica achicar la matriz con esa técnica y obtener una con una dimensión mucho más chica.

REVISAR EL CONCEPTO DE SVD

## Word2Vec

La idea de word2vec es que el significado de mi palabra queda representado por un vector $w_t$ relacionado con la probabilidad de que aparezca esa palabra y un contexto. Entonces, como el contexto de la palabra va a determinar el signicado de la misma, puedo decir dos cosas: o bien es probable que una serie de palabras aparezcan el contexto de mi palabra (modelo Skip-gram), o bien es probable que en presencia de una serie de palabras, aparezca mi palabra (modelo CBOW). Esto se traduce en hacer ML para $P(C|w_t)$ en el primero y hacer ML para $P(w_t|C)$ en el segundo ($C$ serían las palabras del contexto).

En el desarrollo de las cuentas (que lo hace con softmax) encuentra interpretaciones de qué significa cada término (VER VIDEO LECTURE 1, MINUTO 1:05). 

## GloVe

Y después está GloVe, que se supone que hace los dos métodos a la vez.